In [2]:
from tia.bbg import LocalTerminal
import tia.bbg.datamgr as dm
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import LinearSegmentedColormap
import numpy as np
import textwrap
from pandas import ExcelWriter
import openpyxl
from openpyxl.utils import get_column_letter

mgr = dm.BbgDataManager()

In [3]:
import bql
bq = bql.Service()

universe = bq.univ.members(type='AIM', accounts=['BSIOPJ2F','BSIOPJVF',	'BMITSIOF',	'BSIOFJUF',	'BMTCSTCF',	
                                                 'BCTNDLEF',	'BHEWLETF',	'BMJPSF1F',	'BMJPSF2F',	'BWIIOPFF',	
                                                 'BKPIFSSF',	'BKRSSTIF',	'BKRSSTPF',	'BINCMJ1F',	'BMITSTRF',	
                                                 'BCOLFPPF',	'BOHIOHSF',	'BCHICAGF',	'BTCSFTCF',	'BSTFIMUF',	
                                                 'BNPFAACF',	'BSMBCSIF',	'BSTRIMTF',	'BSTRIPFF',	'BCDNUTLF',	
                                                 'BSYMBNDF',	'BGFIPMTF',	'BUSTRMTF',	'BFINPROF',	'BSTRXPFF',	
                                                 'BVBALMTF',	'BMDDIVIF',	'BMDBONDF',	'BMDCLTBF',	'BMDPCBFF',	
                                                 'BMDCDNPF',	'BMDSTBDF',	'BSBYDMTF',	'BUSBYMTF',	'BMDSTRIF',	
                                                 'BSINGMTF',	'BMPFIMPF'], PXNUM='7000')

data_item = bq.data.ts_position()

request = bql.Request(universe, data_item)
response = bq.execute(request)

data = response[0].df()


In [4]:
data.head()

,ACCOUNT,POSITION_ID,SHORT,DATE,STRATEGY,CFD,PRIME_BROKER,BACKEND_DATE,TS_POSITION()
ID,,,,,,,,,
ZL074789 Corp,BMJPSF2F,7294564394944167937,N,2024-02-09,,N,,2024-02-09,180.0
ZL074789 Corp,BMJPSF1F,7294564390649201253,N,2024-02-09,,N,,2024-02-09,450.0
ZQ012847 Corp,BMPFIMPF,6777334268603138064,N,2024-02-09,,N,,2024-02-09,1525.0
EK085421 Corp,BMDSTRIF,7268271687489814804,N,2024-02-09,,N,,2024-02-09,825.0
AW703520 Corp,BMPFIMPF,6647021078192062878,N,2024-02-09,,N,,2024-02-09,4360.0


In [5]:
data = data.groupby('ID').sum()

In [6]:
ids = data.index.to_list()

In [7]:
universe = ids

data_item1 = bq.data.MIFID_SUB_ASSET_CLASS()

request1 = bql.Request(universe, data_item1)


response1 = bq.execute(request1)

data1 = response1[0].df()


In [8]:
mask = data1['MIFID_SUB_ASSET_CLASS()'].str.contains('CRPB', case=False, na=False)

In [9]:
data1 = data1[mask]

In [12]:
ids = data1.index.to_list()

In [61]:
resp = LocalTerminal.get_reference_data(ids,["SECURITY_NAME","MARKET_SECTOR_DES","CAST_PARENT_NAME","CRNCY","IS_EMERGING_MARKET"],ignore_field_error=1)
tickers = resp.as_frame()

In [62]:
issuers = ['BAC']

In [63]:
tickers['issuers']=tickers['SECURITY_NAME'].str.split().str[0]

In [64]:
tickers = tickers[tickers['issuers'].isin(issuers)]

In [65]:
data1 = data1.merge(data, how='left', left_index=True, right_index=True)


In [66]:
tickers = tickers.merge(data1,how='left', left_index=True, right_index=True)



In [67]:
tickers.dropna(subset=['CAST_PARENT_NAME'], inplace=True)


In [68]:
tickers['issuer + currency'] = tickers['CAST_PARENT_NAME'] + tickers['CRNCY']
tickers.drop_duplicates(subset=['issuer + currency'], inplace = True)

In [69]:
tickers = tickers[tickers.IS_EMERGING_MARKET == 'N']
tickers

,SECURITY_NAME,MARKET_SECTOR_DES,CAST_PARENT_NAME,CRNCY,IS_EMERGING_MARKET,issuers,MIFID_SUB_ASSET_CLASS(),TS_POSITION(),issuer + currency
EG133814 Corp,BAC 6.11 01/29/37,Corp,Bank of America Corp,USD,N,BAC,CRPB,3308.0,Bank of America CorpUSD
AN290126 Corp,BAC 1.776 05/04/27,Corp,Bank of America Corp,EUR,N,BAC,CRPB,28090.0,Bank of America CorpEUR


In [24]:
# sample_tickers = tickers[tickers.CRNCY == 'EUR']

In [25]:
# sample_tickers

,SECURITY_NAME,MARKET_SECTOR_DES,CAST_PARENT_NAME,CRNCY,IS_EMERGING_MARKET,issuers,MIFID_SUB_ASSET_CLASS(),TS_POSITION(),issuer + currency
AN290126 Corp,BAC 1.776 05/04/27,Corp,Bank of America Corp,EUR,N,BAC,CRPB,28090.0,Bank of America CorpEUR
AO407361 Corp,TMO 1.4 01/23/26,Corp,Thermo Fisher Scientific Inc,EUR,N,TMO,CRPB,5862.0,Thermo Fisher Scientific IncEUR
BN671028 Corp,GS 0 1/4 01/26/28,Corp,Goldman Sachs Group Inc/The,EUR,N,GS,CRPB,10455.0,Goldman Sachs Group Inc/TheEUR


In [70]:
unique_tickers = tickers.index.to_list()
unique_tickers

['EG133814 Corp', 'AN290126 Corp']

In [71]:
bonds = []
for ticker in unique_tickers:
    universe = bq.univ.filter(bq.univ.bonds(ticker),bq.func.and_(bq.data.crncy() == tickers.loc[ticker]['CRNCY'] ,bq.data.AMT_OUTSTANDING() > 150000000))
    
    data_item = bq.data.name()
    
    request = bql.Request(universe, data_item)
    response = bq.execute(request)

    fetch = response[0].df()
    bonds.append(fetch)

In [73]:
final_dfs = []

for df in bonds:
    ids = df.index.tolist()
    resp = LocalTerminal.get_reference_data(ids,["ID_ISIN", "SECURITY_NAME", "CPN", "MATURITY", "AMT_OUTSTANDING", "YAS_MOD_DUR",
                                                "CNVX_MID", "YLD_CNV_MID", "YLD_CUR_MID", "YAS_ISPREAD_TO_GOVT", "YAS_OAS_SPRD", "LAST_PRICE", "CHG_PCT_YTD","YLD_CHG_YTD_MID_NET",
                                                 "RTG_SP", "RTG_MOODY", "RTG_FITCH", "NXT_CALL_DT", "NXT_CALL_PX", "CRNCY","NORMALIZED_PAYMENT_RANK"],ignore_field_error=1)
    resp_df = resp.as_frame()
    
    df = df.join(resp_df)
    df = df.sort_values(by='MATURITY', ascending=True)
    
    final_dfs.append(df)

In [74]:
final_dfs_merged = []
for df in final_dfs:
    df = df.merge(data, how='left',left_index = True, right_index = True)
    move_col = df.pop('TS_POSITION()')
    df.insert(4,'TS_POSITION()',move_col)
    # df['ROOM_TO_10%'] = df["AMT_OUTSTANDING"]
    # move_col1 = df.pop('ROOM_TO_10%')
    # df.insert(8,'ROOM_TO_10',move_col)

    
    
    final_dfs_merged.append(df)

In [77]:
final_dfs_merged

[                           NAME()       ID_ISIN       SECURITY_NAME      CPN  \
 ID                                                                             
 EK151017 Corp      BAC 4 04/01/24  US06051GFF19      BAC 4 04/01/24  4.00000   
 BP968423 Corp  BAC Float 06/14/24  US06051GJY61  BAC Float 06/14/24  5.76453   
 EK453075 Corp    BAC 4.2 08/26/24  US06051GFH74    BAC 4.2 08/26/24  4.20000   
 EK706228 Corp      BAC 4 01/22/25  US06051GFM69      BAC 4 01/22/25  4.00000   
 AX646307 Corp  BAC 3.458 03/15/25  US06051GHR39  BAC 3.458 03/15/25  3.45800   
 ...                           ...           ...                 ...      ...   
 EJ689431 Corp      BAC Float PERP  US060505ED21      BAC Float PERP  8.77370   
 JK379529 Corp        BAC 6.3 PERP  US060505EU46        BAC 6.3 PERP  6.30000   
 AZ206403 Corp        BAC 5 ⅛ PERP  US060505FP42      BAC 5 1/8 PERP  5.12500   
 553107AE Corp      BAC Float PERP  US553107AE38      BAC Float PERP  0.00000   
 553107AB Corp      BAC Floa

In [ ]:
writer = pd.ExcelWriter('usd_exposure.xlsx', engine='openpyxl')
for df in final_dfs_merged:
    name = df['NAME()'][0].split(" ")[0] + df['CRNCY']
    df.to_excel(writer, sheet_name=name, index=False)

    worksheet = writer.sheets[name]

    for col in df.columns:
        # Find the maximum length of the data in each column
        max_length = max(df[col].astype(str).apply(len).max(), len(col))
        # Adjust the column width; add a small buffer to ensure fit
        column_letter = get_column_letter(df.columns.get_loc(col) + 1)
        worksheet.column_dimensions[column_letter].width = max_length + 2

writer.save()